In [2]:
import numpy as np
np.set_printoptions(precision=2)

room_temp = 18.333
side_length = 10
oven_temp = 162.778
#Target value is 85

brownie_density = 500
brownie_specific_heat = 2550
brownie_conductivity = 0.15 * 3
brownie_alpha = brownie_conductivity / (brownie_density * brownie_specific_heat)

steel_density = 7832
steel_specific_heat = 434
steel_conductivity = 59
steel_alpha = steel_conductivity / (steel_density * steel_specific_heat)

print(brownie_alpha)
print(steel_alpha)

def initialize_brownie_pan(side_length, room_temp):
    return np.full((side_length, side_length), room_temp)


3.529411764705882e-07
1.7357597096632978e-05


In [1]:
def heat_transfer_over_increment(brownie_pan: np.array, dx, dt):
    time_derivative = np.zeros(brownie_pan.shape)
    for x in range(side_length):
        for y in range(side_length):
            # edge
            if (x == 0 and y == 0): # top left corner
                outside_squares = 2 * oven_temp 
                surrounding_squares = brownie_pan[x][y+1] + brownie_pan[x+1][y] + outside_squares
                curr_square = 4 * brownie_pan[x][y]
            elif (x == 0 and y == (side_length - 1)): # bottom left corner
                outside_squares = 2 * oven_temp 
                surrounding_squares = brownie_pan[x][y-1] + brownie_pan[x+1][y] + outside_squares
                curr_square = 4 * brownie_pan[x][y]
            elif (x == (side_length - 1) and y == 0): # top right corner
                outside_squares = 2 * oven_temp 
                surrounding_squares = brownie_pan[x][y+1] + brownie_pan[x-1][y] + outside_squares
                curr_square = 4 * brownie_pan[x][y]
            elif (x == (side_length - 1) and y == (side_length - 1)): # bottom right corner
                outside_squares = 2 * oven_temp 
                surrounding_squares = brownie_pan[x][y-1] + brownie_pan[x-1][y] + outside_squares
                curr_square = 4 * brownie_pan[x][y]
            elif x == 0: # left edge
                surrounding_squares = brownie_pan[x][y+1] + brownie_pan[x][y-1] + brownie_pan[x+1][y] + oven_temp
                curr_square = 4 * brownie_pan[x][y]
            elif y == 0: # top edge
                surrounding_squares = brownie_pan[x+1][y] + brownie_pan[x-1][y] + brownie_pan[x][y+1] + oven_temp
                curr_square = 4 * brownie_pan[x][y]
            elif x == (side_length - 1): # right edge
                surrounding_squares = brownie_pan[x][y+1] + brownie_pan[x][y-1] + brownie_pan[x-1][y] + oven_temp
                curr_square = 4 * brownie_pan[x][y]
            elif y == (side_length - 1): # bottom edge
                surrounding_squares = brownie_pan[x+1][y] + brownie_pan[x-1][y] + brownie_pan[x][y-1] + oven_temp
                curr_square = 4 * brownie_pan[x][y]
            else:
                surrounding_squares = brownie_pan[x + 1][y] + brownie_pan[x - 1][y] + brownie_pan[x][y + 1] + brownie_pan[x][y - 1]
                curr_square = 4 * brownie_pan[x][y]

            time_derivative[x][y] = (brownie_alpha / (dx ** 2)) * (surrounding_squares - curr_square)

    brownie_pan += (time_derivative * dt)
    return brownie_pan

brownie_pan = initialize_brownie_pan(side_length, room_temp)
for i in range(130000):
    brownie_pan = heat_transfer_over_increment(brownie_pan, dx=0.025, dt=0.1) # 0.01 meters (1cm), 0.01 seconds
    if brownie_pan[int(side_length/2)][int(side_length/2)] >= 85:
        print(f"{i / 100 / 60} minutes of cooking")
        break

print(brownie_pan)

NameError: name 'np' is not defined